# Exercise 20

## Neuronale Netzwerke

Viele Fortschritte im Bereich AI wurden durch neuronale Netzwerke erzielt. Ein einfaches neuronales Netzwerk hast du übrigens bereits gesehen, nämlich die lineare Regression! Ich zeige dir zuerst wie die lineare Regression mit der Pytorch library für neuronale Netzwerke funktioniert:

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Diese Funktion erstellt Dummy Daten für unser Experiment
def create_dummy_data(size=10000):
    # Mit einm festen seed bekommen wir immer die gleichen "zufälligen" Daten
    np.random.seed(42)
    x = np.random.normal(size=size)
    y = 2 + x + np.random.normal(size=size, scale=0.1)
    
    return np.resize(x, (size, 1)), np.resize(y, (size, 1))

# Hier definieren wir nun das Modell in Pytorch
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()

        # Hier definieren wir das "neuronale Netzwerk".
        # Wir haben hier eine lineare Layer mit den Argumenten 1, 1.
        # Das heisst wir haben einen Input und einen Output.
        # Im Hintergrund multipliziert die Layer die Inputzahl mit einem Gewicht.
        # Während dem Training lernt die Layer ihr Gewicht!
        self.linear_layer = nn.Linear(1, 1)
    
    # Die forward Funktion beschreibt nun, wie wir aus dem Input den Output generieren
    def forward(self, x):
        return self.linear_layer(x)

def linear_regression_with_pytorch():
    x, y = create_dummy_data()
    
    # Hier wandeln wir die Daten in Tensoren um, dass ist einfach die Pytorch Version von Arrays :).
    x, y = torch.FloatTensor(x), torch.FloatTensor(y)
    
    # Hier erstellen wir nun das Modell, welches wir oben definiert haben.
    model = LinearRegressionModel()
    
    # Hier definieren wir den Loss, also den Fehler welchen wir minimieren möchten.
    loss_function = nn.MSELoss()
    
    # Zusätzlich brauchen wir noch einen Optimizer.
    # Dieser passt die Modellparameter an.
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
    # Nun beginnen wir mit dem Training, wir gehen in "Epochen" vor.
    # In jeder Epoche trainieren wir einmal über die ganzen Trainingsdaten.
    for epoch in range(1000):
        # Hier setzten wir die Gradienten des Optimizers auf 0.
        # Kannst du dich an eine Übung erinnern, in welcher wir Steigung einer Kurve berechnet haben um das Minimum zu finden :)?
        optimizer.zero_grad()
        
        # Wir berchnen nun die Vorhersage des neuronalen Netzwerks.
        prediction = model.forward(x)
        
        # Hier schauen wir wie weit wir daneben lagen.
        loss = loss_function(prediction, y)
        
        # Hier wird nun berechnet, wie wir die Modellparameter anpassen müssen, um den Loss kleiner zu machen.
        loss.backward()
        
        # Hier passt der Optimizer unsere Parameter an.
        optimizer.step()
        
        if epoch % 100 == 0:
            print(f'Wir sind in Epoche {epoch + 1}, der Loss ist {loss.item()} und die Modellparameter sind:')
            print(dict(model.named_parameters()))

linear_regression_with_pytorch()

### Aufgabe 1

Kannst du diese Fragen beantworten?

1. Betrachte zuerst `create_dummy_data`, verstehst du was für Daten diese Funktion generiert? Falls du willst kannst du einen Plot machen um es besser zu verstehen.
2. Kannst du grob erklären, wie unser neuronales Netzwerk aufgebaut ist und was es macht? Vielleicht helfen dir die Kommentare, vielleicht auch einfach ein Youtube Video.
3. Verstehst du den Output des Trainings? Was passiert mit dem Loss und den Parametern?
4. Was denkst du, welche Methode verwendet Pytorch um den Loss zu minimieren? Du hast diese Methode auch schon kennengelernt.
5. Verstehst du was die beiden Parameter `linear_layer.weight` und `linear_layer.bias` bedeuten?

### Aufgabe 2

Eine der wichtigsten Eigenschaften eines neuronalen Netzwerks ist seine Architektur. Denn nur mit der richtigen Architektur kann das neuronale effektiv trainiert werden! Wir möchten nun untersuchen, wie die Architektur den Output des neuronalen Netzwerks beeinflusst. Ich gebe dir folgende Funktionen zur  Hilfe:

In [ ]:
import matplotlib.pyplot as plt

def evaluate_neural_net_performance(Model, data='linear', data_size=1000, epochs=10):
    def linear_func(x):
        return 2 + x
    
    def nonlinear_func(x):
        return [8 * (v - 0.5)**2 + 4 * v * np.sin(20 * v * v) for v in x]
    
    data_creation_func = linear_func if data == 'linear' else nonlinear_func

    np.random.seed(42)
    x = np.random.uniform(size=data_size)
    y = data_creation_func(x) + np.random.normal(size=data_size, scale=0.05)
    
    x, y = torch.FloatTensor(np.resize(x, (data_size, 1))), torch.FloatTensor(np.resize(y, (data_size, 1)))

    model = Model()
    
    loss_function = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        optimizer.zero_grad()

        prediction = model.forward(x)
        loss = loss_function(prediction, y)
        loss.backward()
        optimizer.step()
    
    print(f'Final loss: {loss.item()}')

    x_predict = torch.FloatTensor(np.reshape(np.linspace(0, 1, 100), (100, 1)))
    prediction = model.forward(x_predict)
    
    plt.scatter(x, y, label= 'Training Data')
    plt.plot(x_predict, prediction.detach().numpy().flatten(), c='r', label='Neural Net')
    plt.plot(x_predict, data_creation_func(x_predict), c='g', label='Ground Truth')
    plt.ylabel('y')
    plt.xlabel('x')
    plt.legend()
    plt.show()

In [ ]:
evaluate_neural_net_performance(LinearRegressionModel, data='linear', data_size=1000, epochs=10000)

Untersuche nun, wie gut `LinearRegressionModel` mit `data='nonlinear'` funktioniert.

In [ ]:
...

Was stellst du fest?

### Aufgabe 3

Hier ist nun ein Modell mit zwei Layers:

In [ ]:
class TwoLayerModel(nn.Module):
    def __init__(self):
        super(TwoLayerModel, self).__init__()
        
        # Die "mittlere" Layer hat hier 2 Knoten!
        # Wichtig ist nur, dass man mit 1 beginnt und mit 1 endet, denn wir haben 1-dimensionalen Input und Output.
        self.linear_layer_1 = nn.Linear(1, 2)
        self.linear_layer_2 = nn.Linear(2, 1)
    
    def forward(self, x):
        out = self.linear_layer_1(x)
        out = self.linear_layer_2(out)
        
        return out

Untersuche die Performance dieses Modells auf den linearen und nicht-linearen Daten.

In [ ]:
...

### Aufgabe 4

Hier ist ein Modell mit einer zwei Layer und einer Aktivierung:

In [ ]:
class TwoLayerWithActivationModel(nn.Module):
    def __init__(self):
        super(TwoLayerWithActivationModel, self).__init__()
        
        self.linear_layer_1 = nn.Linear(1, 4)
        self.linear_layer_2 = nn.Linear(4, 1)
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.linear_layer_1(x)
        out = self.relu(out)
        out = self.linear_layer_2(out)
        
        return out

Untersuche die Performance diese Modells auf den linearen und nicht-linearen Daten. Findest du heraus, was die ReLu Funktion macht?

In [ ]:
...

### Aufgabe 5

Erstelle nun ein Modell um auch die non-linearen Daten zu fitten. Du kannst sowohl mehr Layers hinzufügen, als auch die Layers breiter machen. Verwende zwischen den Layers jeweils ReLu. Tipp: Du musst dein Netzwerk zumindest ein bisschen tiefer (=mehr Layers), aber sehr viel breiter machen.

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        ...
    
    def forward(self, x):
        ...
        
        return out

In [ ]:
...